# Lecture 5: SBOX CPA Attack

## Auxiliary definitions

In [1]:
import sys
sys.path.append('..')

In [2]:
import pandas as pd
import bokeh
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper

output_notebook()

Loading BokehJS ...

In [3]:
import numpy as np

SBOX = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76, # 0
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0, # 1
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15, # 2
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75, # 3
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84, # 4
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf, # 5
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8, # 6
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2, # 7
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73, # 8
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb, # 9
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79, # a
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08, # b
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a, # c
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e, # d
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf, # e
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16  # f
]

def sbox(data):
    return SBOX[data]

sbox_vec = np.vectorize(sbox)

In [4]:
import numpy as np

HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    if isinstance(n, str):
        return HW[ord(n)]
    return HW[n]

hw_vec = np.vectorize(hw)

def pearson(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    return sum((x - x_mean) * (y - y_mean)) / np.sqrt(sum((x - x_mean) ** 2) * sum((y - y_mean) ** 2))

In [5]:
def pearson_pointwise(traces, intermediates):
    intermediates_diff = intermediates - np.mean(intermediates)
    intermediates_sqrt = np.sqrt(np.sum(intermediates_diff ** 2))
    traces_diff = traces - np.mean(traces, axis=0)
    
    return np.sum(traces_diff * intermediates_diff[:, None], axis=0) / (
        np.sqrt(np.sum(traces_diff ** 2, axis=0)) * intermediates_sqrt
    )

In [6]:
import securec
from securec import util
scope, target = util.init()

See https://chipwhisperer.readthedocs.io/en/latest/api.html#firmware-update


In [7]:
import numpy as np

scope.default_setup()

secret_key = [0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B, 0x0C, 0x0D, 0x0E, 0x0F, 0x10]

def capture(cmd, pt, samples=500, randfunc=lambda: random.randbytes(16), check_output=True):
    scope.adc.samples = samples
    scope.arm()
    target.simpleserial_write(cmd, pt + randfunc())
    traces = np.array(util.capture())
    if check_output:
        output = np.array(target.simpleserial_read(0x00))
        expected_output = [sbox(secret_key[i] ^ inp) for i, inp in enumerate(pt)] #sbox_vec(secret_key ^ list(pt))
        assert np.all(output == expected_output), f"Wrong result: {output} != {expected_output}" 
    return traces

In [8]:
import random
import tqdm
import tqdm.notebook

def capture_traces_random_input(
    cmd, 
    num_traces, 
    num_samples=500,
    randfunc=lambda: bytes([random.randint(0, 255) for _ in range(16)]),
    check_output=False,
):
    traces = []
    inputs = []
    for _ in tqdm.notebook.tqdm(range(num_traces)):
        input = bytes([random.randint(0, 255) for _ in range(16)])
        traces.append(capture(cmd, input, samples=num_samples, randfunc=randfunc, check_output=check_output))
        inputs.append(input)
    traces = np.array(traces)
    attempts = np.array([list(a) for a in inputs])
    return traces, attempts

In [9]:
import math


def pearson_pointwise_multi(traces, intermediates):
    (n, t) = traces.shape
    (_, m) = intermediates.shape

    d_traces = traces - np.einsum(
        "nt->t", traces, dtype="float64", optimize="optimal"
    ) / np.double(n)
    d_intermediates = intermediates - np.einsum(
        "nm->m", intermediates, dtype="float64", optimize="optimal"
    ) / np.double(n)

    tmp1 = np.einsum("nm,nm->m", d_intermediates, d_intermediates, optimize="optimal")
    tmp2 = np.einsum("nt,nt->t", d_traces, d_traces, optimize="optimal")
    tmp = np.einsum("m,t->mt", tmp1, tmp2, optimize="optimal")
    denominator = np.sqrt(tmp)
    numerator = np.einsum("nm,nt->mt", d_intermediates, d_traces, optimize="optimal")

    return np.nan_to_num(numerator / denominator)


def plot_correlation_vs_traces(
    traces,
    attempts,
    key_index=0,
    plotpoints=20,
):
    # Compute data
    plotpoints = min(plotpoints, len(traces))
    data = np.zeros((256, plotpoints))
    intermediates = np.array(
        [hw_vec(sbox_vec(attempts[:, key_index] ^ guess)) for guess in range(256)]
    ).T
    for i in range(0, plotpoints):
        j = math.ceil((i + 1) / plotpoints * len(traces))
        data[:, i] = np.max(
            np.abs(pearson_pointwise_multi(traces[:j, :], intermediates[:j, :])), axis=1
        )

    source = {
        "xs": len(data) * [list(range(0, len(traces), math.ceil(len(traces) / plotpoints)))],
        "ys": [corr for corr in data],
        "legend": list(range(256)),
        "color": (math.ceil(256 / 20) * bokeh.palettes.Category20_20)[:256],
    }
    correlations = np.sort(data[:, -1])
    # Create figure
    p = figure(
        sizing_mode="stretch_width",
        height=300,
        tooltips=[("char", "@legend"), ("corrleation", "$y")],
        y_range=(0, 1)
    )
    p.multi_line(xs="xs", ys="ys", color="color", source=source)
    p.add_layout(
        bokeh.models.Title(
            text=f"correlation ratio = {1 - correlations[-2] / correlations[-1]:.3f}",
            text_font_size="12pt",
        ),
        "above",
    )
    p.add_layout(
        bokeh.models.Title(
            text=f"max correlation = {correlations[-1]:.3f}", text_font_size="12pt"
        ),
        "above",
    )
    p.add_layout(
        bokeh.models.Title(text="Correlation vs Traces", text_font_size="16pt"), "above"
    )
    return p


In [10]:
securec.util.compile_and_flash('./5_sbox_solution.c')

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2929 bytes
✓


## Variant 1: Plain SBOX lookup

```c
for (uint8_t i = 0; i < 16; i++)
{
    data[i] = aes_sbox[aes_key[i] ^ buf[i]];
}
```

In [11]:
show(plot_correlation_vs_traces(*capture_traces_random_input(0x01, 500)))

  0%|          | 0/500 [00:00<?, ?it/s]

## Variant 2: SBOX lookup in randomized order

```c
uint8_t rand = random[0] & 0x0F;
for (uint8_t i = 0; i < 16; i++)
{
    data[i ^ rand] = aes_sbox[aes_key[i ^ rand] ^ buf[i ^ rand]];
}
```

### Variant 2.1 Randomize with 1 bit

By controlling entropy of random we can regulate how many different orders can be reached. Let's start with 1 bit randomness, i.e. `rand = 0` or `rand = 1`.
This means we have 2 different possibilities for the loop:
- `rand = 0`: `0 -> 1 -> 2 -> ... -> 15`
- `rand = 1`: `1 -> 0 -> 3 -> ... -> 14`

In [12]:
show(plot_correlation_vs_traces(*capture_traces_random_input(
    0x02,
    500,
    randfunc=lambda: bytes([random.randint(0, 255) & 0x01 for _ in range(16)]),
    check_output=False,
)))

  0%|          | 0/500 [00:00<?, ?it/s]

### Variant 2.2 Randomize with 2 bits

With `rand = 0, 1, 2, 3` we can get 4 different loop orders.

In [13]:
show(plot_correlation_vs_traces(*capture_traces_random_input(
    0x02,
    5000,
    randfunc=lambda: bytes([random.randint(0, 255) & 0x03 for _ in range(16)]),
    check_output=False,
)))

  0%|          | 0/5000 [00:00<?, ?it/s]

### Variant 2.3 Randomize with 3 bits

With `rand = 0, 1, 2, 3, 4, 5, 6, 7` we can get 8 different loop orders.

In [14]:
show(plot_correlation_vs_traces(*capture_traces_random_input(
    0x02,
    10000,
    randfunc=lambda: bytes([random.randint(0, 255) & 0x07 for _ in range(16)]),
    check_output=False,
)))

  0%|          | 0/10000 [00:00<?, ?it/s]

We can observe that the maximal correlation _halves_ whenever we add one more bit of randomness. Therefore, if we want to reach the same correlation ratio, we have to _double_ the number of traces recorded.

### Variant 2.4 Randomize with 4 bits

With `rand = 0, ..., 15` we have reached the maximal number of different variants.

In [15]:
show(plot_correlation_vs_traces(*capture_traces_random_input(
    0x02,
    10000,
    randfunc=lambda: bytes([random.randint(0, 255) & 0x0f for _ in range(16)]),
    check_output=False,
)))

  0%|          | 0/10000 [00:00<?, ?it/s]

## Variant 3: Insert timing jitter

Correlation power analysis relies on the fact that points-of-interest (e.g. the SBOX-lookup) for different inputs happens always at the same time.

```c
uint8_t rand = random[0];
for (uint8_t i = 0; i < 16; i++)
{
    volatile uint8_t j = 0;
    while (j < rand)
    {
        j++;
    }
    data[i] = aes_sbox[aes_key[i] ^ buf[i]];
}
```

### Variant 3.1 Insert 1 bit timing jitter

In [16]:
show(plot_correlation_vs_traces(*capture_traces_random_input(
    0x03,
    5000,
    randfunc=lambda: bytes([random.randint(0, 255) & 0x01 for _ in range(16)]),
    check_output=False,
)))

  0%|          | 0/5000 [00:00<?, ?it/s]

### Variant 3.2 Insert 2 bit timing jitter

In [17]:
show(plot_correlation_vs_traces(*capture_traces_random_input(
    0x03,
    20000,
    randfunc=lambda: bytes([random.randint(0, 255) & 0x03 for _ in range(16)]),
    check_output=False,
)))

  0%|          | 0/20000 [00:00<?, ?it/s]

## Variant 4: Dummy lookup

Another really good and scalable method to push correlation down is using dummy lookups. I.e. we processing the lookups in random order as on variant 2 but inject lookups with random data.

Assuming `buf` holds at least 32 bytes we can just write:

```c
uint8_t output[32];
uint8_t rand = random[0];
for (uint8_t i = 0; i < 32; i++)
{
    output[i ^ rand] = aes_sbox[aes_key[i ^ rand] ^ buf[i ^ rand]];
}
```

In [18]:
show(plot_correlation_vs_traces(*capture_traces_random_input(
    0x04,
    30000,
    randfunc=lambda: bytes([random.randint(0, 255) & 0x1f for _ in range(16)]),
    check_output=False,
)))

  0%|          | 0/30000 [00:00<?, ?it/s]

Compare this result with Variant 2.4! 

## Variant 5: Randomized SBOX 

The ultimate resort is an randomized SBOX. That means you have a mask $m_o$ and a modified AES-SBOX $\mathrm{SBOX'}$ with the property:

$$ \mathrm{SBOX'}(\mathrm{pt}_i \oplus k_i) = \mathrm{SBOX}(\mathrm{pt}_i \oplus k_i) \oplus m_o $$

Where $k_i$ is the $i$-th key byte and $\mathrm{pt}_i$ the $i$-th input byte.

But, randomizing the SBOX is expensive! 

In [19]:
show(plot_correlation_vs_traces(*capture_traces_random_input(
    0x05,
    100000,
    randfunc=lambda: bytes([random.randint(0, 255) for _ in range(16)]),
    check_output=False,
)))

  0%|          | 0/100000 [00:00<?, ?it/s]

Note: 
- This method can be extended to protect also the first xor ($\mathrm{pt}_i \oplus k_i$) by masking the key with a mask $m_i$.
- In general, the code creating the masked SBOX has to be protected against side-channel attacks.
- Usually the SBOX is re-masked after a certain amount of usages. 